#### Imports

In [1038]:
import pandas as pd
import numpy as np
import re

#### Read linkedin csv

In [1039]:
linkedin = pd.read_csv('../data/linkedin.csv')
linkedin.sample(3)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
353,3772901664,"Data Research Analyst, gt.school (Remote) - $6...",Crossover,"· Austin, TX",30 $/hr,En remoto,Jornada completa,Algo de responsabilidad,De 5.001 a 10.000 empleados · Servicios y cons...
425,3775180566,DATA AND PROCESS ANALYST,University of Washington,"· Seattle, WA",6.482 $/mes - 8.102 $/mes,Presencial,Jornada completa,Intermedio,Más de 10.001 empleados · Enseñanza superior
64,3744009051,RF Computational Engineer 3,Lam Research,"· Fremont, CA",67.733 $/año - 91.440 $/año,En remoto,Jornada completa,Intermedio,Más de 10.001 empleados · Fabricación de semic...


#### Cleaning 'company_state' column

In [1040]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'NC', 'SC', 'CO', 'CT', 'ND', 'SD', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
          'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NJ', 'NY', 'NH', 'NM', 'OH', 'OK', 'OR', 'PA', 'RI', 'TN', 'TX', 'UT',
          'UT', 'VT', 'VA', 'WV', 'WA', 'WI', 'WY']
linkedin['company_state'] = linkedin['company_state'].apply(lambda row: 'Estados Unidos' if row == '· Estados Unidos' else row)
linkedin['company_state'].value_counts()

company_state
Estados Unidos         58
· Nueva York, NY       22
· Seattle, WA          16
· San Francisco, CA    16
· Washington, DC       10
                       ..
· Plano, TX             1
· Lexington, KY         1
· Montpelier, VT        1
· Bolingbrook, IL       1
· Scott AFB, IL         1
Name: count, Length: 216, dtype: int64

In [1041]:
def fclean_company_state(df, states):
    for i in states:
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: i if i in str(row) else row)
    return df

linkedin = fclean_company_state(linkedin, states)

In [1042]:
def sclean_company_state(df, states):
    for key, value in states.items():
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: value if key in str(row) else row)
    return df

left_states = {
    'Virginia': 'VA',
    'Minnesota': 'MN',
    'Washington': 'WA',
    'San Francisco': 'CA',
    'Nebraska': 'NE',
    'California': 'CA',
    'Texas': 'TX',
    'Nueva York': 'NY',
    'Michigan': 'MI',
    'Luisiana': 'LA',
    'Florida': 'FL',
    'Maryland': 'MD',
    'Nevada': 'NV',
    'Oregón': 'OR',
    'Oklahoma': 'OK',
    'Georgia': 'GA',
    'Carolina del Sur': 'SC',
    'Delaware': 'DE',
    'Ohio': 'OH',
    'Arkansas': 'AR',
    'Iowa': 'IA',
    'Carolina del Norte': 'NC',
    'Indiana': 'IN',
    'Nashville': 'TN',
    'Massachusetts': 'MA',
    'Tennessee': 'TN',
    'Enid': 'OK'
}
linkedin = sclean_company_state(linkedin, left_states)

#### Cleaning 'company_state' column

In [1043]:
def clean_remote_ratio(df, dict_):
    for key, val in dict_.items():
        df['remote_ratio'] = df['remote_ratio'].apply(lambda row: key if key in str(row) else str(row))
    return df

remote_dict = {
    'Presencial': 0,
    'Híbrido': 50,
    'En remoto': 100
}
linkedin = clean_remote_ratio(linkedin, remote_dict)
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     232
Híbrido       135
Presencial    129
nan            11
Name: count, dtype: int64

In [1044]:
linkedin.loc[linkedin['remote_ratio'] == 'nan', 'remote_ratio'] = np.nan
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     232
Híbrido       135
Presencial    129
Name: count, dtype: int64

#### Cleaning 'experience_level' column

In [1045]:
linkedin.sample(5)
linkedin['experience_level'].value_counts()

experience_level
Intermedio                 238
Sin experiencia            130
Prácticas                   65
Algo de responsabilidad     59
Director                     7
Ejecutivo                    2
Name: count, dtype: int64

#### Cleaning 'employment_type' column

In [1046]:
def clean_employment_type(df, list_):
    for i in list_:
        df['employment_type'] = df['employment_type'].apply(lambda row: i if i in str(row) else str(row))
    return df

remote_dict = [
    'Jornada completa',
    'Contrato por obra',
    'Media jornada',
    'Prácticas'
]
linkedin = clean_employment_type(linkedin, remote_dict)
linkedin['employment_type'].value_counts()

employment_type
Jornada completa     327
Contrato por obra     84
Prácticas             55
Media jornada         41
Name: count, dtype: int64

#### Cleaning 'company_size' column

In [1047]:
linkedin['company_size'].value_counts()

company_size
De 51 a 200 empleados · Servicios y consultoría de TI                    34
Más de 10.001 empleados · Hospitales y atención sanitaria                24
De 51 a 200 empleados · Dotación y selección de personal                 21
De 1.001 a 5.000 empleados · Servicios y consultoría de TI               21
De 1.001 a 5.000 empleados · Tecnología, información e internet          19
                                                                         ..
De 51 a 200 empleados · Servicios de bienestar y mantenimiento físico     1
De 51 a 200 empleados · Hospitales y atención sanitaria                   1
De 201 a 500 empleados · Relaciones gubernamentales                       1
De 1.001 a 5.000 empleados · Servicios públicos                           1
De 51 a 200 empleados · Servicios de publicidad                           1
Name: count, Length: 138, dtype: int64

In [1048]:
linkedin['employees'] = linkedin['company_size'].apply(lambda row: row.split('·')[0].strip())
linkedin['employees'].value_counts()

employees
Más de 10.001 empleados                                                     111
De 51 a 200 empleados                                                        99
De 1.001 a 5.000 empleados                                                   91
De 11 a 50 empleados                                                         48
De 201 a 500 empleados                                                       43
De 501 a 1.000 empleados                                                     43
De 5.001 a 10.000 empleados                                                  40
Entre 1 y 10 empleados                                                       28
Ve una comparación con los otros 300 solicitantes. Probar Premium gratis      1
Fabricación de productos de plástico                                          1
Ve una comparación con los otros 444 solicitantes. Probar Premium gratis      1
Servicios para el individuo y la familia                                      1
Name: count, dtype: int64

In [1049]:
# Patrones que deseas buscar en la columna específica
patrones = [
    r"Ve una comparación con los otros \d+ solicitantes\. Probar Premium gratis",
    r"Fabricación de productos de plástico",
    r"Servicios para el individuo y la familia"
]

# Crear una expresión regular combinando los patrones con el operador OR (|)
regex_pattern = '|'.join(patrones)

# Filtrar el DataFrame por la columna específica
linkedin[linkedin['employees'].str.contains(regex_pattern)]

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,employees
155,3762242381,Data Scientist,Probar Premium gratis,IL,140.000 $/año - 170.000 $/año,Híbrido,Jornada completa,Intermedio,Ve una comparación con los otros 300 solicitan...,Ve una comparación con los otros 300 solicitan...
335,3771514066,Quality Control Analyst,"Carolina Poly, Inc.",SC,65.000 $/año - 72.000 $/año,Presencial,Jornada completa,Sin experiencia,Fabricación de productos de plástico,Fabricación de productos de plástico
427,3775200323,Data Analyst,Probar Premium gratis,MA,35 $/h - 40 $/h,En remoto,Media jornada,NaN,Ve una comparación con los otros 444 solicitan...,Ve una comparación con los otros 444 solicitan...
450,3775649749,Data Consultant ( Remote),Fresca Fresh Blanket Service,Estados Unidos,NaN,En remoto,Media jornada,Algo de responsabilidad,Servicios para el individuo y la familia,Servicios para el individuo y la familia


In [1050]:
linkedin.loc[linkedin['employees'].str.contains(regex_pattern), 'employees'] = np.nan
linkedin['employees'].value_counts()

employees
Más de 10.001 empleados        111
De 51 a 200 empleados           99
De 1.001 a 5.000 empleados      91
De 11 a 50 empleados            48
De 201 a 500 empleados          43
De 501 a 1.000 empleados        43
De 5.001 a 10.000 empleados     40
Entre 1 y 10 empleados          28
Name: count, dtype: int64

In [1051]:
pattern1 = r"Más de (\d+\.?\d*) empleados"
pattern2 = r"De (\d+\.?\d*) a (\d+\.?\d*) empleados"
pattern3 = r"Entre (\d+\.?\d*) y (\d+\.?\d*) empleados"
def clean_employee(row):
    match1 = re.match(pattern1, str(row))
    match2 = re.match(pattern2, str(row))
    match3 = re.match(pattern3, str(row))
    if match1:
        return match1.group(1)
    elif match2:
        return match2.group(2)
    elif match3:
        return match3.group(2)

linkedin['employees'] = linkedin['employees'].apply(clean_employee)
linkedin['employees'].value_counts()

employees
10.001    111
200        99
5.000      91
50         48
500        43
1.000      43
10.000     40
10         28
Name: count, dtype: int64

In [1052]:
def standarize_company_size(row):
    if pd.notnull(row):
        if int(row.replace('.', '')) < 50:
            return 'S'
        elif int(row.replace('.', '')) <= 250:
            return 'M'
        elif int(row.replace('.', '')) > 250:
            return 'L'
        else:
            return np.nan

linkedin['company_size'] = linkedin['employees'].apply(standarize_company_size)
linkedin['company_size'].value_counts()

company_size
L    328
M    147
S     28
Name: count, dtype: int64

#### Cleaning 'salary_range' column

In [1053]:
linkedin['salary_range'].unique()

array(['100.000 $/año - 150.000 $/año', '283.780 $/año - 331.640 $/año',
       '145.000 $/año - 180.000 $/año', '95.000 $/año - 130.000 $/año',
       '167.200 $/año - 250.800 $/año', '200.000 $/año - 240.000 $/año',
       '70.000 $/yr', nan, '120.000 $/año - 220.000 $/año',
       '144.000 $/año - 270.250 $/año', '170.112 $/año - 237.000 $/año',
       '170.000 $/año - 190.000 $/año', '20 $/hr',
       '67.733 $/año - 91.440 $/año', '95.000 $/año - 120.000 $/año',
       '78 $/h - 104 $/h', '122.000 $/año - 190.000 $/año',
       '48 $/h - 65 $/h', '45 $/h - 55 $/h', '40 $/h - 43 $/h',
       '90.000 $/año - 130.000 $/año', '70.000 $/año - 85.000 $/año',
       '150.000 $/año - 200.000 $/año', '26 $/h - 31 $/h',
       '139.100 $/año - 235.400 $/año', '60.000 $/año - 100.000 $/año',
       '140.000 $/año - 170.000 $/año', '90.000 $/año - 110.000 $/año',
       '400.000 $/año - 700.000 $/año', '140.000 $/año - 165.000 $/año',
       '73.500 $/año - 170.000 $/año', '87.600 $/año - 118

In [1054]:
hour_pattern = r"(\d+) \$\/hr"
year_pattern = r"(\d+\.?\d*) \$\/yr"
hour_range_pattern = r"(\d+\,?\d*) \$\/h - (\d+\,?\d*) \$\/h"
year_range_pattern = r"(\d+\.?\d*) \$\/año - (\d+\.?\d*) \$\/año"
month_range_pattern = r"(\d+\.?\d*) \$\/mes - (\d+\.?\d*) \$\/mes"


def clean_salary(row):
    match_h = re.match(hour_pattern, str(row))
    match_y = re.match(year_pattern, str(row))
    match_rh = re.match(hour_range_pattern, str(row))
    match_ry = re.match(year_range_pattern, str(row))
    match_rm = re.match(month_range_pattern, str(row))
    if match_h:
        num_h = match_h.group(1)
        num_w = int(num_h) * 40
        return int(num_w * 48)
    elif match_y:
        return int(match_y.group(1).replace('.', ''))
    elif match_rh:
        min_ = match_rh.group(1)
        max_ = match_rh.group(2)
        num_h = float(min_.replace(',', '.')) + float(max_.replace(',', '.')) / 2
        num_w = num_h * 40
        return int(num_w * 48)
    elif match_ry:
        min_ = match_ry.group(1)
        max_ = match_ry.group(2)
        num_y = float(min_.replace('.', '')) + float(max_.replace('.', '')) / 2
        return int(num_y)
    elif match_rm:
        min_ = match_rm.group(1)
        max_ = match_rm.group(2)
        num_m = float(min_.replace('.', '')) + float(max_.replace('.', '')) / 2
        return int(num_m * 12)
    else:
        return np.nan

linkedin['salary_range'] = linkedin['salary_range'].apply(clean_salary)
linkedin['salary_range'].value_counts()

salary_range
230000.0    22
225000.0    21
207750.0    21
279125.0    18
57600.0     18
            ..
208000.0     1
107500.0     1
16205.0      1
265000.0     1
154560.0     1
Name: count, Length: 90, dtype: int64

In [1055]:
linkedin.sample(5)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,employees
476,3776486316,Data Engineer,Dice,Estados Unidos,139200.0,En remoto,Jornada completa,Sin experiencia,L,5.000
475,3776484734,AI Engineer,Motion Recruitment,WA,130000.0,En remoto,Jornada completa,Sin experiencia,L,1.000
213,3766632486,Project Data Analyst (F&B),The Walt Disney Company,FL,49920.0,Híbrido,Prácticas,Prácticas,L,10.001
95,3755065806,Data Engineer Intern,Hewlett Packard Enterprise,CA,NaN,Híbrido,Prácticas,Prácticas,L,10.001
94,3755047295,Data Analyst,Ledgent Technology,FL,NaN,Híbrido,Prácticas,Prácticas,L,1.000


#### Cleaning 'job_title' column

In [1056]:
linkedin['job_title'].value_counts()

job_title
Data Engineer                                                                                            42
Data Analyst                                                                                             42
Data Scientist                                                                                           33
Machine Learning Engineer                                                                                25
Data Analyst  I                                                                                          19
                                                                                                         ..
Azure Cloud Data Engineer                                                                                 1
Data Engineer/Lead - Dallas, NY/NJ, Chicago, IL, Atlanta, GA (Onsite for now but could be hybrid too)     1
Junior Data Analyst - 22325                                                                               1
Discovery Data Sci

In [1062]:
def standarize_job(row):
    title = str(row).lower()
    if 'data engineer' in title:
        return 'Data Engineer'
    elif 'data analyst' in title:
        return 'Data Analyst'
    elif 'data scientist' in title:
        return 'Data Scientist'
    elif 'data' in title and 'junior' not in title:
        if 'engineer' in title:
            return 'Data Engineer'
        elif 'analyst' in title:
            return 'Data Analyst'
    elif 'machine' in title and 'learning' in title:
        return 'Machine Learning Engineer'
    elif 'junior' in title:
        if 'data' in title and 'engineer' in title:
            return 'Data Engineer'
        elif 'data' in title and 'analyst' in title:
            return 'Data Analyst'
    elif 'business' in title:
        if 'intelligence' in title and 'analyst' in title:
            return 'BI Analyst'
        elif 'analyst' in title:
            return 'Business Analyst'
        else:
            return 'BI Engineer'
    elif 'artificial intelligence' in title or 'ai' in title:
        print(title)
        if 'research engineer' in title:
            return 'Research Engineer'
        elif 'ml' in title:
            return 'Machine Learning Engineer'
        elif 'engineer' in title:
            return 'AI Engineer'
        else:
            return title
    else:
        return str(row)

linkedin['job_title'] = linkedin['job_title'].apply(standarize_job)
linkedin['job_title'].value_counts()

ai engineer
ai engineer
summer intern - r&d/artificial intelligence
ai engineer
senior paid social analyst (growth marketing team)
responsible ai testing science consultant
ai engineer
artificial intelligence architecture(fully remote)
ai engineer
solutions architect, generative ai
ai engineer


job_title
Data Analyst                                  127
Data Engineer                                 116
Data Scientist                                113
Machine Learning Engineer                      50
None                                           20
                                             ... 
Remote Resort Reservationist - Entry Level      1
Remote Marketing Travel Specialist              1
CYBER INTEL ANALYST - PEN TESTER                1
Remote Coordinator - Entry Level                1
solutions architect, generative ai              1
Name: count, Length: 71, dtype: int64

In [1058]:
linkedin.to_csv('../data/linkedin_standarized.csv', index=False)